In [1]:
#importing all the important library or module which is used in this project
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [2]:
#loading the fashion amnist dataset
data = pd.read_csv('fmnist_small.csv')
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [3]:
#if i want to see the target variable counts
data.label.value_counts()  #this case come under the multi classifier category

label
6    656
7    620
5    612
2    604
4    595
0    595
3    591
1    590
8    582
9    555
Name: count, dtype: int64

In [4]:
#splitting the data into train tets split
from sklearn.model_selection import train_test_split

x = data.drop("label",axis=1)
y = data['label']

In [5]:
#checking gpu avilable in system or not
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
#now splitting them into training and testing set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [7]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape,x_train.ndim,x_test.ndim,y_train.ndim,y_test.ndim

((4800, 784), (1200, 784), (4800,), (1200,), 2, 2, 1, 1)

In [8]:
x_train.pixel1.min(),x_train.pixel1.max()

(np.int64(0), np.int64(0))

In [9]:
#now doing input scaling and applying min max scaling to inputs
x_train = x_train/255.0
x_test  = x_test/255.0

In [10]:
type(x_train),x_train.iloc[0].dtype

(pandas.core.frame.DataFrame, dtype('float64'))

In [11]:
#changing training and testing dataset to numpy array object
x_train_np = np.array(x_train)
x_test_np =  np.array(x_test)
y_train_np = np.array(y_train)
y_test_np =  np.array(y_test)

In [12]:
x_train_np.ndim,x_train_np.dtype,x_test_np.ndim,x_test_np.dtype,y_train_np.ndim,y_train_np.dtype,y_test_np.ndim,y_test_np.dtype

(2,
 dtype('float64'),
 2,
 dtype('float64'),
 1,
 dtype('int64'),
 1,
 dtype('int64'))

# in case of multi classifier pblm statement inputs tensor dtype will be float32 and output varibale will be long

In [13]:
#now converting pandas input and output df objects  to tensor object
x_train_tensor = torch.tensor(x_train_np,dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_np,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np,dtype=torch.long)
y_test_tensor = torch.tensor(y_test_np,dtype=torch.long)

In [14]:
x_train_tensor.dtype,x_test_tensor.dtype,y_train_tensor.dtype,y_test_tensor.dtype

(torch.float32, torch.float32, torch.int64, torch.int64)

# using dataset class i am creating custom class that split data into batch size

In [15]:
class CustomDataset(Dataset):
    #creating constructor method in that we r initializing the instance variable in it.
    def __init__(self,input_data,output_data):
        #calling parent class property
        super().__init__()
        self.input_data = input_data
        self.output_data = output_data
        
        
    #using another magical method which show the shape of input data
    def __len__(self):
        return len(self.input_data)
    
    #using another magical method which divide data into batches based on index
    def __getitem__(self, index):
        return self.input_data[index],self.output_data[index]

In [16]:
#now calling the customdatset class
training_dataset = CustomDataset(x_train_tensor,y_train_tensor)
testing_dataset  = CustomDataset(x_test_tensor,y_test_tensor)

# now using dataloader class to convert dataset into batches

In [17]:
train_dataloader_set = DataLoader(training_dataset,batch_size=8,shuffle=True,pin_memory=True)
test_dataloader_set = DataLoader(testing_dataset,batch_size=8,shuffle=True,pin_memory=True)

In [18]:
for feature,label in train_dataloader_set:
    print(feature)
    print(label)
    break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([6, 2, 7, 0, 8, 9, 6, 9])


In [19]:
x_train_tensor.shape

torch.Size([4800, 784])

In [20]:
data.label.nunique()

10

# defining the Neural network fully connected layer

In [21]:
from torch.nn import Sequential

class MyNeuralNetworkLayer(nn.Module):
    
    #using constructor method for defining the architecture of neural network !!!
    def __init__(self,input_feature_shape):
        #calling the parent class constructor inheriting the property from it.
        super().__init__()
        
        self.model = Sequential(
            
            #defining the 1st hidden layer neurons
            nn.Linear(input_feature_shape,512), #index=0
            nn.BatchNorm1d(512),                #index=1
            nn.ReLU(),
            nn.Dropout(0.3),
            
            
            #efining the 2nd hidden layer neurons
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            #defining the 3rd hidden layer neurons
            nn.Linear(256,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            #defining the output layer
            nn.Linear(64,10)
            #NOTE: No Softmax here → CrossEntropyLoss expects raw logits(value automatically they find probability of each class)
         
        )
    
    #creating another method to excute propogation class
    def forward(self,input_feature):
        return self.model(input_feature)

In [22]:
#creating an object of custom neural network class
inp_shape = x_train_tensor.shape[1]
model = MyNeuralNetworkLayer(inp_shape).to(device=device)
model

MyNeuralNetworkLayer(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=256, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [23]:
#if i wan tto show how many no of trainable parameter trained by model
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
MyNeuralNetworkLayer                     --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       401,920
│    └─BatchNorm1d: 2-2                  1,024
│    └─ReLU: 2-3                         --
│    └─Dropout: 2-4                      --
│    └─Linear: 2-5                       131,328
│    └─BatchNorm1d: 2-6                  512
│    └─ReLU: 2-7                         --
│    └─Dropout: 2-8                      --
│    └─Linear: 2-9                       16,448
│    └─BatchNorm1d: 2-10                 128
│    └─ReLU: 2-11                        --
│    └─Dropout: 2-12                     --
│    └─Linear: 2-13                      650
Total params: 552,010
Trainable params: 552,010
Non-trainable params: 0

In [24]:
#now defining how many no of iteration want to give at time of training,what will be loss function,optimers

loss_fxn = nn.CrossEntropyLoss() #this loss fxn will be same as sparse categorical cross entropy
optimizers = torch.optim.Adam(model.parameters(),lr=0.0001,weight_decay=0.01) 
#optimizer we used to update the trainable paramerter in each layer during Back Propgation time
epochs=10

In [25]:
optimizers

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.01
)

In [26]:
print("First Hidden Layer Weights (Before Training):")
print(model.model[0].weight)

First Hidden Layer Weights (Before Training):
Parameter containing:
tensor([[ 0.0271, -0.0071, -0.0040,  ...,  0.0129, -0.0144,  0.0207],
        [ 0.0140,  0.0227,  0.0017,  ...,  0.0035,  0.0179, -0.0064],
        [-0.0345,  0.0214,  0.0244,  ..., -0.0094, -0.0069,  0.0234],
        ...,
        [ 0.0164,  0.0279, -0.0294,  ...,  0.0072,  0.0211, -0.0310],
        [ 0.0260, -0.0078, -0.0230,  ..., -0.0297,  0.0195, -0.0353],
        [ 0.0168,  0.0049,  0.0249,  ...,  0.0215, -0.0236,  0.0333]],
       device='cuda:0', requires_grad=True)


In [27]:
print("First Hidden Layer Bias (Before Training):")
print(model.model[0].bias)

First Hidden Layer Bias (Before Training):
Parameter containing:
tensor([ 3.3911e-02,  2.0725e-02,  1.7302e-02,  5.1094e-03,  3.3677e-02,
         2.0735e-02, -1.6800e-02, -2.5497e-02,  3.4659e-02, -1.3791e-02,
         1.9636e-02,  1.4516e-02,  3.5280e-02, -2.4202e-02,  8.4557e-05,
         4.4624e-03, -1.8843e-02,  3.2147e-04, -2.8055e-02,  3.3293e-03,
         3.5492e-02, -2.3488e-02,  2.3243e-02,  2.7403e-02, -2.8101e-02,
        -2.7987e-02, -6.0804e-03,  1.9627e-02,  3.2127e-02,  2.3716e-02,
        -2.4373e-02,  1.2310e-02,  1.2037e-02, -1.9227e-02,  1.7759e-03,
        -3.1030e-02, -1.3256e-03,  5.2897e-03,  2.9442e-02,  1.9674e-03,
        -1.8112e-02, -9.9332e-03, -1.2363e-02,  2.1835e-02, -2.7482e-02,
        -3.4528e-02,  1.6416e-02, -1.9818e-02, -8.9446e-03, -8.5720e-03,
        -1.4880e-02,  5.1100e-03, -2.8110e-02, -2.6748e-02, -1.2635e-02,
        -1.9693e-02, -1.5282e-03, -1.0389e-02, -8.5818e-03, -2.0200e-02,
         2.8962e-02,  2.3448e-02,  2.3464e-02, -2.0666e-02,

In [28]:
print("Second Hidden Layer Weights (Before Training):")
print(model.model[4].weight)

Second Hidden Layer Weights (Before Training):
Parameter containing:
tensor([[-0.0433,  0.0329, -0.0310,  ...,  0.0242, -0.0415,  0.0125],
        [ 0.0218,  0.0034, -0.0213,  ..., -0.0008,  0.0246,  0.0236],
        [ 0.0213,  0.0060,  0.0358,  ..., -0.0267, -0.0245,  0.0074],
        ...,
        [ 0.0019,  0.0061,  0.0198,  ..., -0.0307,  0.0228,  0.0339],
        [ 0.0014,  0.0092, -0.0069,  ..., -0.0142, -0.0409,  0.0024],
        [-0.0181, -0.0049,  0.0316,  ..., -0.0082,  0.0239, -0.0070]],
       device='cuda:0', requires_grad=True)


# now we will start the training of model

In [29]:
avg_training_loss,avg_testing_loss = [],[]
for epoch in range(epochs):
    model.train()
    
    training_loss_count = 0  # <- Reset for each epoch
    
    #first we load tha dataset and passing them into neural architecture 
    for train_feature,train_label in train_dataloader_set:
        #now laoding the data into GPU
        train_feature = train_feature.to(device)
        train_label   = train_label.to(device)
        
        #now passing the training feature to neural netowrk architecture the first algorithm that work is forward pass.
        #generate the prediction output
        y_prediction = model.forward(train_feature)
        
        #evaluating the performance of model at time of training by calculating the loss value.
        losses = loss_fxn(y_prediction,train_label)
        
        #updating the training_loss_count
        training_loss_count = training_loss_count+losses.item()
        
        #clearing the gradient in optimizers object
        optimizers.zero_grad() #before reducing the loss value by using backpropgation algorithm first of we have to clear the gradients which present in optimizers
        
        #now reducing the loss value by doing back propgation algorithm
        losses.backward() #back propgation algorithm will calculate the gradients in each layer of trainaianble parameter wrt to losses
        
        #updating the trainable parameter by using optimizers
        optimizers.step()
    
    #now calculating average training loss.
    avg_train_loss = training_loss_count/len(train_dataloader_set)
    avg_training_loss.append(avg_train_loss)
    
    
    
    #now validating the testing loss or testing data.
    testing_loss_count = 0
    
    model.eval()
    
    #while testing the data we have to stop trcking the require gradient not applying overfitting and backpropogation in it.
    with torch.no_grad():
        for test_feature,test_label in test_dataloader_set:
            #now loading the data into GPU
            test_feature = test_feature.to(device)
            test_label   = test_label.to(device)
            
            #now passing the test feature into neural network architecture and doing forward pass calculation.
            prediction = model.forward(test_feature) #return each batch ke andar each record ke 10 probability
            
            #calculating the testing loss and updating the testing_loss_count
            test_loss = loss_fxn(prediction,test_label)
            
            testing_loss_count = testing_loss_count+test_loss.item()
    
        #now calculating average training loss.
        avg_test_loss = testing_loss_count/len(test_dataloader_set)
        avg_testing_loss.append(avg_test_loss)
        
        
        
        print(f"Epoch [{epoch+1}/{epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Test Loss: {avg_test_loss:.4f}")
           

Epoch [1/10], Train Loss: 1.6350, Test Loss: 1.1678
Epoch [2/10], Train Loss: 1.2235, Test Loss: 0.9319
Epoch [3/10], Train Loss: 1.0522, Test Loss: 0.8252
Epoch [4/10], Train Loss: 0.9556, Test Loss: 0.7138
Epoch [5/10], Train Loss: 0.8661, Test Loss: 0.6981
Epoch [6/10], Train Loss: 0.8183, Test Loss: 0.6334
Epoch [7/10], Train Loss: 0.7698, Test Loss: 0.6148
Epoch [8/10], Train Loss: 0.7408, Test Loss: 0.5886
Epoch [9/10], Train Loss: 0.7062, Test Loss: 0.5989
Epoch [10/10], Train Loss: 0.6954, Test Loss: 0.5510


# to track train accuracy and test accuracy while training and evaluating the model

In [ ]:
avg_training_loss, avg_testing_loss = [], []
training_accuracies, testing_accuracies = [], []

for epoch in range(epochs):
    # ---- TRAINING ----
    model.train()
    training_loss_count = 0
    correct_train = 0
    total_train = 0
    
    for train_feature, train_label in train_dataloader_set:
        train_feature = train_feature.to(device)
        train_label   = train_label.to(device)
        
        # Forward pass
        y_prediction = model(train_feature)
        
        # Loss
        losses = loss_fxn(y_prediction, train_label)
        training_loss_count += losses.item()
        
        # Accuracy calculation (for multiclass)
        _, predicted = torch.max(y_prediction, 1)  #_, predicted = torch.max(output, 1) → highest probability class index nikalta hai.
        correct_train += (predicted == train_label).sum().item()
        total_train   += train_label.size(0)
        
        # Backpropagation
        optimizers.zero_grad()
        losses.backward()
        optimizers.step()
    
    avg_train_loss = training_loss_count / len(train_dataloader_set)
    train_accuracy = (correct_train / total_train) * 100 #accuracy training calculation
    avg_training_loss.append(avg_train_loss)
    training_accuracies.append(train_accuracy)
    
    # ---- TESTING ----
    model.eval()
    testing_loss_count = 0
    correct_test = 0
    total_test = 0
    
    with torch.no_grad():
        for test_feature, test_label in test_dataloader_set:
            test_feature = test_feature.to(device)
            test_label   = test_label.to(device)
            
            prediction = model(test_feature)
            test_loss = loss_fxn(prediction, test_label)
            testing_loss_count += test_loss.item()
            
            # Accuracy calculation
            _, predicted = torch.max(prediction, 1)
            correct_test += (predicted == test_label).sum().item()
            total_test   += test_label.size(0)
    
    avg_test_loss = testing_loss_count / len(test_dataloader_set)
    test_accuracy = (correct_test / total_test) * 100
    avg_testing_loss.append(avg_test_loss)
    testing_accuracies.append(test_accuracy)
    
    print(f"Epoch [{epoch+1}/{epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% "
          f"| Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.2f}%")


Epoch [1/10], Train Loss: 0.6885, Train Acc: 79.08% | Test Loss: 0.5779, Test Acc: 81.42%
Epoch [2/10], Train Loss: 0.6715, Train Acc: 79.08% | Test Loss: 0.5574, Test Acc: 80.83%
Epoch [3/10], Train Loss: 0.6500, Train Acc: 80.19% | Test Loss: 0.5291, Test Acc: 83.25%
Epoch [4/10], Train Loss: 0.6259, Train Acc: 81.31% | Test Loss: 0.5259, Test Acc: 82.33%
Epoch [5/10], Train Loss: 0.6195, Train Acc: 81.10% | Test Loss: 0.5251, Test Acc: 82.75%
Epoch [6/10], Train Loss: 0.6113, Train Acc: 80.98% | Test Loss: 0.5488, Test Acc: 81.17%
Epoch [7/10], Train Loss: 0.6031, Train Acc: 81.21% | Test Loss: 0.5409, Test Acc: 83.00%
Epoch [8/10], Train Loss: 0.6117, Train Acc: 81.40% | Test Loss: 0.5131, Test Acc: 83.08%
Epoch [9/10], Train Loss: 0.5908, Train Acc: 81.92% | Test Loss: 0.5329, Test Acc: 81.58%
Epoch [10/10], Train Loss: 0.5706, Train Acc: 83.06% | Test Loss: 0.5110, Test Acc: 83.83%
